# Testing DNase_HiC Libraries
MiSeq run of HiC libraries prepared with protocols.io link here....

Tried method on:
- Kiwifruit (*Actinidia chinensis*)
- *Eucalyptus regnans*
- Karaka (*Corynocarpus laevigatus*)
- Grape (*Vitis sp.*)
- *Wahlenbergia ceracea*


### Metadata
Input genomes here:
/workspace/hraijc/HiC_trials/Ash_MinION/Genomes

HiC MiSeq Data here:
/input/genomic/plant/Actinidia/PopulationsStudies/MiSeq/ExperimentRequestor10984/AG1002_dataset/

- Data Sequenced by Auckland Genomics (AG1002)
- Experiment requestor number 10984

In [29]:
# Set Global Variables
WKDIR=/workspace/hraijc/HiC_trials/Ash_MinION
TEMP=${WKDIR}/temp
LOGDIR=${WKDIR}/log
GENOMEDIR=${WKDIR}/Genomes
HiCDIR=/input/genomic/plant/Actinidia/PopulationsStudies/MiSeq/ExperimentRequestor10984/AG1002_dataset
HiC_CLEAN=${WKDIR}/HiC_Clean

In [30]:
# Set up workspace
mkdir -p $LOGDIR
mkdir -p $TEMP
mkdir -p $HiC_CLEAN


In [31]:
cd ${WKDIR}

In [ ]:
#create reference files for assemblies.

module load samtools/0.1.19

for ASSEMBLY in ${GENOMEDIR}/*.fasta
do
    samtools faidx $ASSEMBLY
    cut -f1,2 ${ASSEMBLY}.fai > ${ASSEMBLY%.fasta}.genome
done | abatch -j ${LOGDIR}/AssemblyPrep --time 00:10:00 --mem 1G | sbatch



## Wahlenbergia

In [4]:
#Set Variables
SPDIR=${WKDIR}/Wahlenbergia
mkdir -p $SPDIR
ASSEMBLY=${GENOMEDIR}/wahlenbergia_ceracea_unphased.fasta
APREFIX=Wahlenbergia_hic
HiC_RAW=${HiCDIR}/Wahlenbergia-Hi-C_S4_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz


## Eucalyptus

In [6]:
#Set Variables
SPDIR=${WKDIR}/Eucalyptus
mkdir -p $SPDIR
ASSEMBLY=${GENOMEDIR}/E_regnans_unphased.fasta
APREFIX=Eucalyptus_hic
HiC_RAW=${HiCDIR}/Eucalyptus-Hi-C_S1_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz


## Grape

In [36]:
#Set Variables
SPDIR=${WKDIR}/Grape
mkdir -p $SPDIR
ASSEMBLY=${GENOMEDIR}/VITVvi_vZin03_v1.1.primary.fasta
APREFIX=Grape_hic
HiC_RAW=${HiCDIR}/Grape-Hi-C_S2_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz


## Karaka

In [10]:
#Set Variablessss
SPDIR=${WKDIR}/Karaka
mkdir -p $SPDIR
ASSEMBLY=${GENOMEDIR}/karaka_phasing.hic.hap1.p_ctg.fasta
APREFIX=Karaka_hic
HiC_RAW=${HiCDIR}/Karaka-5-Hi-C_S3_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz


## Kiwifruit

In [12]:
#Set Variables
SPDIR=${WKDIR}/Kiwifruit
mkdir -p $SPDIR
ASSEMBLY=${GENOMEDIR}/XX_V2a.fasta
APREFIX=Kiwifruit_hic
HiC_RAW=${HiCDIR}/TC-T2-Hi-C-XX_S7_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz


In [26]:
#Set Variables
SPDIR=${WKDIR}/Kiwifruit_XX
mkdir -p $SPDIR
ASSEMBLY=/workspace/hraijc/Kiwi/XX/HiC8_Novaseq/RefGenome/CK69_01_v2_contigs_min1kb.fasta
APREFIX=Kiwifruit_XX_hic
HiC_RAW=/workspace/hraijc/Kiwi/XX/HiC8_MiSeq/220622_M01815_0436/HiC8-lib_S1_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz

In [27]:
du -sh ${ASSEMBLY}

662M	/workspace/hraijc/Kiwi/Ck69_01_monoploid/HiC8_Novaseq/RefGenome/CK69_01_v2_contigs_min1kb.fasta


## Raspberry

In [34]:
#Set Variables
SPDIR=${WKDIR}/Wakefield
mkdir -p $SPDIR
ASSEMBLY=${GENOMEDIR}/Wakefield.fasta
APREFIX=Wakefield_hic
HiC_RAW=/input/genomic/plant/Rubus/idaeus/Wakefield_genome/HIC/MiseqRun/RI-Hi-C_S2_L001_R
READ1=${HiC_RAW}1_001.fastq.gz
READ2=${HiC_RAW}2_001.fastq.gz

In [37]:
#Run HiC workflow

sbatch << EOF
#!/bin/bash
#SBATCH -J hic
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=8
#SBATCH --mem=10G
#SBATCH --time=12:00:00

module load bwa/0.7.17
module load samtools/0.1.19
module load conda
module load fastp/0.23.2


cd ${SPDIR}

### Clean HiC Reads with fastp.###
fastp \
-i ${READ1} \
-o ${HiC_CLEAN}/${APREFIX}_Clean_R1.fastq.gz \
-I ${READ2} \
-O ${HiC_CLEAN}/${APREFIX}_Clean_R2.fastq.gz \
--trim_front1 15 \
--trim_front2 15 \
--qualified_quality_phred 20 \
--length_required 50 \
--thread 8


###Align reads with BWA. Use -5SP for Hi-C reads.###
bwa index $ASSEMBLY
bwa mem -5SP -t8 ${ASSEMBLY} ${HiC_CLEAN}/${APREFIX}_Clean_R1.fastq.gz ${HiC_CLEAN}/${APREFIX}_Clean_R2.fastq.gz -o ${TEMP}/${APREFIX}.sam



###Flag PCR Duplicates with SAMBLASTER###
/workspace/hraijc/git_clones/samblaster/samblaster -i ${TEMP}/${APREFIX}.sam -o ${TEMP}/${APREFIX}_marked_byread.sam



### Remove PCR Duplicates### 
samtools view -S -b -h -@ 8 -F 2316 ${TEMP}/${APREFIX}_marked_byread.sam > ${APREFIX}_dedup.bam



### HiC QC from Phase genomics ###
conda activate hraijc_hic_qc2
python /workspace/hraijc/git_clones/hic_qc/hic_qc.py -n -1 -b ${APREFIX}_dedup.bam --outfile_prefix ${APREFIX}
conda deactivate



### Run YAHS ###
/workspace/hraijc/git_clones/yahs/yahs $ASSEMBLY ${APREFIX}_dedup.bam -o ${APREFIX}_yahs



### YAHS without error correction. ###
/workspace/hraijc/git_clones/yahs/yahs --no-contig-ec --no-scaffold-ec $ASSEMBLY ${APREFIX}_dedup.bam -o ${APREFIX}_yahsNoEC




EOF


Submitted batch job 876797
